In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from os import listdir
import cv2
import numpy as np
import pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import VGG16
from keras.layers import Input, Flatten, Dense, Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import random
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
#cd /content/drive/MyDrive/Colab Notebooks/NienLuanNganh

/content/drive/MyDrive/Colab Notebooks/NienLuanNganh


In [ ]:
raw_folder = "./money_data/"
def save_data(raw_folder=raw_folder):
    print("Bắt đầu xử lý ảnh...")

    pixels = []
    labels = []

    # Lặp qua các folder con trong thư mục raw
    for folder in listdir(raw_folder):
        if folder!='.DS_Store':
            print("Folder=",folder)
            # Lặp qua các file trong từng thư mục chứa các em
            for file in listdir(raw_folder  + folder):
                if file!='.DS_Store':
                    print("File=", file)
                    pixels.append( cv2.resize(cv2.imread(raw_folder  + folder +"/" + file),dsize=(224,224)))
                    labels.append( folder)

    pixels = np.array(pixels)
    labels = np.array(labels)#.reshape(-1,1)

    from sklearn.preprocessing import LabelBinarizer
    encoder = LabelBinarizer()
    labels = encoder.fit_transform(labels)
    print(labels)

    file = open('moneydata.data', 'wb')
    # dump information to that file
    pickle.dump((pixels,labels), file)
    # close the file
    file.close()

    return

In [ ]:
save_data()

In [ ]:
def load_data():
    file = open('./moneydata.data', 'rb')

    # dump information to that file
    (pixels, labels) = pickle.load(file)

    # close the file
    file.close()

    print(pixels.shape)
    print(labels.shape)

    return pixels, labels

In [ ]:
X,y = load_data()

(738, 224, 224, 3)
(738, 7)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=100)

In [ ]:
y_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0]])

In [ ]:
model_vgg16_conv = VGG16(weights='imagenet', 
                         include_top=False)

58900480/58889256 [==============================] - 1s 0us/step


In [ ]:
# Dong bang cac layer
for layer in model_vgg16_conv.layers:
  layer.trainable = False

In [ ]:
# Tao model
input = Input(shape=(224,224,3), name='image_input')
output_vgg16_conv = model_vgg16_conv(input)

# Them cac layer FC va Dropout
x = Flatten(name='flatten')(output_vgg16_conv)
x = Dense(4096, activation='relu', name='fc1')(x)
x = Dropout(0.5)(x)
x = Dense(4096, activation='relu', name='fc2')(x)
x = Dropout(0.5)(x)
x = Dense(7, activation='softmax', name='predictions')(x)

# Compile
vggmodel = Model(inputs=input, outputs=x)
vggmodel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
#vggmodel.summary()

In [ ]:
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.1,
    rescale=1./255,
	width_shift_range=0.1,
    height_shift_range=0.1,
	horizontal_flip=True,
    brightness_range=[0.2,1.5], fill_mode="nearest")

In [ ]:
aug_val = ImageDataGenerator(rescale=1./255)

In [ ]:
filepath="weights-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
vgghist=vggmodel.fit_generator(aug.flow(X_train, y_train, batch_size=64),
                               epochs=30,# steps_per_epoch=len(X_train)//64,
                               validation_data=aug.flow(X_test,y_test,
                               batch_size=len(X_test)),
                               callbacks=callbacks_list)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
10/10 [==============================] - 15s 2s/step - loss: 0.5362 - accuracy: 0.7932 - val_loss: 0.2733 - val_accuracy: 0.9189

Epoch 00001: val_accuracy did not improve from 0.98649
Epoch 2/30
10/10 [==============================] - 13s 1s/step - loss: 0.5078 - accuracy: 0.7949 - val_loss: 0.2198 - val_accuracy: 0.9392

Epoch 00002: val_accuracy did not improve from 0.98649
Epoch 3/30
10/10 [==============================] - 13s 1s/step - loss: 0.4722 - accuracy: 0.8119 - val_loss: 0.1709 - val_accuracy: 0.9595

Epoch 00003: val_accuracy did not improve from 0.98649
Epoch 4/30
10/10 [==============================] - 13s 1s/step - loss: 0.4804 - accuracy: 0.8068 - val_loss: 0.1616 - val_accuracy: 0.9459

Epoch 00004: val_accuracy did not improve from 0.98649
Epoch 5/30
10/10 [==============================] - 13s 1s/step - loss: 0.4460 - accuracy: 0.8169 - val_loss: 0.1983 - val_accuracy: 0.9662

Epoch 00005: val_accuracy did not improve from 0.98649
Epoch 6/30
10/10 [==